# Nowy Model Lekarz

## Cel Notebooka
Notebook został przygotowany jako przewodnik, który prowadzi od instalacji zależności, przez importy, wczytanie danych, ich przetwarzanie, analizę normalności, budowę modeli regresji, aż po wizualizacje i raportowanie wyników.

## Opis Sekcji
- Instalacja zależności
- Import bibliotek
- Wczytanie i eksploracja danych
- Czyszczenie danych i przekształcenia
- Analiza normalności i diagnostyka
- Modelowanie statystyczne
- Wizualizacje i raportowanie
- Wnioski i rekomendacje

In [4]:
# Instalacja zależności - uruchom tylko przy pierwszym uruchomieniu w nowym środowisku
!pip install -r requirements.txt

In [5]:
# Import wszystkich niezbędnych bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, anderson, normaltest
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [6]:
###########################################
# 4. Czyszczenie danych i przekształcenia  #
###########################################

# Zakładamy, że dane zostały już wczytane do zmiennej 'data' (z sekcji 3)

# Przypisywanie kategorii i zmiana nazw etykiet
data['miejscowosc'] = pd.Categorical(
    data['klasa_miejsca'], 
    categories=[1, 2, 3, 4, 5, 6], 
    ordered=True
)
data['miejscowosc'] = data['miejscowosc'].cat.rename_categories([
    "powyzej 500 tys", "200-499 tys", "100-199 tys", "20-99 tys", "ponizej 20 tys", "wies"
])

# Przekształcenie kolumny 'wyksz' na kategorię
data['wyksztalcenie_1'] = pd.Categorical(
    data['wyksz'], 
    categories=[1, 2, 3], 
    ordered=True
)
data['wyksztalcenie_1'] = data['wyksztalcenie_1'].cat.rename_categories([
    "podstawowe", "srednie", "wyzsze"
])

# Przekształcenie kolumny 'moz_oszcz' na kategorię możliwości oszczędzania
data['mozliwosci_oszczedzania_1'] = pd.Categorical(
    data['moz_oszcz'], 
    categories=[1, 2, 3, 4], 
    ordered=True
)
data['mozliwosci_oszczedzania_1'] = data['mozliwosci_oszczedzania_1'].cat.rename_categories([
    "regularnie", "nieregularnie", "mozemy_ale_nie", "nie_mozemy"
])

# Przekształcenie kolumny 'glowne_zrod_utrzym'
data['zrodlo_utrzymania_1'] = pd.Categorical(
    data['glowne_zrod_utrzym'], 
    categories=[1, 2, 3, 4, 5], 
    ordered=True
)
data['zrodlo_utrzymania_1'] = data['zrodlo_utrzymania_1'].cat.rename_categories([
    "pracownicy", "rolnicy", "praca na wlasny rachunek", "emeryt/rencista", "niezarobkowe zrodla dochodu"
])

# Przekształcenie kolumny 'syt_mat'
data['sytuacja_materialna_1'] = pd.Categorical(
    data['syt_mat'], 
    categories=[1, 2, 3, 4, 5], 
    ordered=True
)
data['sytuacja_materialna_1'] = data['sytuacja_materialna_1'].cat.rename_categories([
    "dobra", "raczej dobra", "przecietna", "raczej zla", "zla"
])

# Filtrowanie danych - usuwamy obserwacje z zerowymi wydatkami na opiekę zdrowotną
model2 = data[data['wyd_os_lek'] > 0].copy()

# Przykładowe podsumowanie statystyczne wydatków w zależności od miejscowości
summary_miejscowosc = model2.groupby('miejscowosc', observed=False)['wyd_os_lek'].describe()
print(summary_miejscowosc)

NameError: name 'data' is not defined

In [ ]:
###############################################################
# 5. Analiza danych – testy normalności i diagnostyka         #
###############################################################

def test_normality(series, col_name):
    """
    Wykonuje testy normalności dla podanej serii danych oraz generuje Q-Q plot.
    """
    values = series.dropna()
    # Test Shapiro-Wilka
    shapiro_stat, shapiro_p = shapiro(values)
    
    # Test Andersona-Darlinga
    anderson_result = anderson(values, dist='norm')
    
    # Test D'Agostino-Pearsona
    dagostino_stat, dagostino_p = normaltest(values)
    
    print(f"\nNormality tests for {col_name}:")
    print(f"Shapiro-Wilk: stat = {shapiro_stat:.4f}, p = {shapiro_p:.4f}")
    print(f"Anderson-Darling: stat = {anderson_result.statistic:.4f}")
    print(f"D'Agostino-Pearson: stat = {dagostino_stat:.4f}, p = {dagostino_p:.4f}")
    
    # Generowanie Q-Q plot
    sm.qqplot(values, line='45', fit=True)
    plt.title(f"Q-Q Plot for {col_name}")
    plt.show()

# Przykładowe wywołanie testu na kolumnie 'wyd_os_lek'
test_normality(model2['wyd_os_lek'], 'wyd_os_lek')

In [ ]:
#############################################
# 6. Modelowanie statystyczne             #
#############################################

# Przygotowanie zmiennej zależnej - obliczamy logarytm z wydatków
model2['log_wydatki'] = np.log(model2['wyd_os_lek'])

# Przygotowanie macierzy predyktorów
# Załóżmy, że w modelu wykorzystamy zmienne: 'dochod_los', 'liczba_osob' oraz 'inwalidztwo'
predictors = ['dochod_los', 'liczba_osob', 'inwalidztwo']
X = model2[predictors]
X = sm.add_constant(X)  # Dodanie stałej do modelu

# Zmienna zależna
y = model2['log_wydatki']

# Budowa modelu OLS
model = sm.OLS(y, X).fit()
print("Podstawowy model OLS:")
print(model.summary())

# Budowa modelu OLS z heteroskedastycznymi błędami (robust standard errors)
robust_model = model.get_robustcov_results(cov_type='HC0')
print("\nModel OLS z heteroskedastycznymi błędami (HC0):")
print(robust_model.summary())

# Przykładowe testy diagnostyczne
# Influence Plot
fig, ax = plt.subplots(figsize=(8,6))
sm.graphics.influence_plot(model, ax=ax, criterion='cooks')
plt.title('Influence Plot')
plt.show()

# Obliczenie VIF (Variance Inflation Factor) dla oceny multikolinearności
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['Variable'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("\nVariance Inflation Factor (VIF):")
print(vif)

In [ ]:
#############################################
# 7. Wizualizacje i raportowanie          #
#############################################

# Residuals vs Fitted Plot
fitted_vals = model.fittedvalues
residuals = model.resid
plt.figure(figsize=(8,6))
plt.scatter(fitted_vals, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()

# Q-Q Plot of Residuals
sm.qqplot(residuals, line='45', fit=True)
plt.title('Q-Q Plot of Residuals')
plt.show()

# Influence Plot
fig, ax = plt.subplots(figsize=(8,6))
sm.graphics.influence_plot(model, ax=ax, criterion='cooks')
plt.title('Influence Plot')
plt.show()

## Wnioski i rekomendacje

Podsumowując przeprowadzoną analizę:
- Model OLS wykazał statystycznie istotny wpływ niektórych zmiennych na logarytm wydatków.
- Testy normalności sugerują, że reszty modelu nie są w pełni zgodne z założeniem normalności, co widoczne jest zarówno w testach, jak i na wykresie Q-Q.
- Wizualizacje (wykresy reszt, influence plot) wskazują na obecność obserwacji odstających, które mogą wpłynąć na wyniki modelowania.

**Rekomendacje:**
- Dalsze badania diagnostyczne, np. robust regression oraz analiza VIF, mogą pomóc w ocenie stabilności modelu.
- Rozważenie transformacji zmiennych lub usunięcia obserwacji odstających w celu poprawy jakości modelu.
- Zastosowanie walidacji krzyżowej do oceny jakości modelu.
- W dalszych analizach warto rozważyć alternatywne metody, takie jak modele nieparametryczne, aby poradzić sobie z odchyleniami od normalności.

Dalsze badania mogą również obejmować dodatkowe testy regresyjne oraz analizy diagnostyczne, które pozwolą lepiej zrozumieć strukturę danych i zależności między zmiennymi.